# <b>Data Diagnostics I </b> *✲ﾟ*｡✧٩(･ิᴗ･ิ๑)۶*✲ﾟ*｡✧

In this notebook we will explore taking the min-max or percentile normalization between datasets and also derivatives and see how our data changes, i.e. the distribution of each variable, check the principal components, etc.